# Learning goals
After today's lesson you should be able to:
- Use the Google Maps API via the `googlemaps` library to: 
    - Find geometry routes between two points
    - Geocode addresses
    - Get points of interest


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd


import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context(context='paper')

os.getcwd()


# 1. Google Cloud

Google Cloud is a set of "cloud computing" services that consists of physical assets like computers and hard drives, and virtual resources, like computing environments, that is often used for data storage and management, tools for computing and software development, machine learning, and more. 

You may have heard of Amazon AWS and Microsoft Azure, these are Amazon and Microsoft's cloud computing offerings that all have similar capabilities. 

To note: These "cloud" computers and services don't live in some ephemeral space. You can actually see the locations of [Google Data Centers](https://www.google.com/about/datacenters/locations/). 

This is what the Dalles, Oregon Google data center looks like from satellite imagery. It's just a giant building filled with a lot of computers:  
<iframe src="https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d1390.42297102674!2d-121.20177632936587!3d45.63191557459542!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x54961ebdb45cdec5%3A0x4bde13efb5371001!2sThe%20Dalles%20Google%20Data%20Center!5e1!3m2!1sen!2sus!4v1676841727765!5m2!1sen!2sus" width="1000" height="600"  style="display: block; margin: 0 auto"  allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>

The Google Maps API is a tool that allows us to use some of the engines and retrieve datasets underlying Google Maps.


## 1.1 Getting started
To get started, we need to **create an API key**. An API key is a code or id provided by the client that is passed to the API that allows it to identify which user/project is calling the API to keep track of your usage and access. It also allows you to keep track of your API usage for particular projects and applications. 

You'll need your Cornell account to proceed. Please refer my Slack message from earlier this week on the details. 

- Go to the [Google Cloud console](https://console.cloud.google.com/), which is your landing page for all Google computing services (geocoding API is one of many tools you can use through the Google Cloud).
- If your default account is your personal gmail (like it is mine), make sure to switch it to your Cornell email on the top right-hand corner.
- You'll have to agree to their terms of service before proceeding (I think). An [interesting read](https://cloud.google.com/terms/?authuser=1) for another time.
- This should take you to a page like this:
</figure>
<img src="https://www.dropbox.com/s/wj9igmb9dhlb3ba/Screen%20Shot%202023-02-19%20at%204.29.43%20PM.png?dl=1" alt="drawing" width="1000" style="display: block; margin: 0 auto"/>
</figure>

- In the upper-left corner, click **Select a project** and **NEW PROJECT**. All API keys are associated with projects, which is more for developers who are using the services more heavily. This is so that we separate usage and keep track of what projects are using how much of which resources.
</figure>
<img src="https://www.dropbox.com/s/vwje4uz06tlvlo9/Screen%20Shot%202023-02-19%20at%204.33.29%20PM.png?dl=1" alt="drawing" width="1000" style="display: block; margin: 0 auto"/>
</figure>

- Here, input a project name. You'll need to create a **globally unique** name, all lower case, no spaces only dashes name. I'm calling this project `introUDS-wx233`. You can call yours `introUDS-YOURNETID`
- Once you click ok it will take a few seconds to create the project.
- In the console, select your new project `introUDS-wx233` (for me).
- From here, go to the sidebar menu, select **Google Maps Platform->APIs**

</figure>
<img src="https://www.dropbox.com/s/ru6b367zd6jwh1l/Screen%20Shot%202023-02-19%20at%204.34.48%20PM.png?dl=1" alt="drawing" width="1000" style="display: block; margin: 0 auto"/>
</figure>

- You will be given an API key. Save that down somewhere. 
- You will also be asked to **Protect your API key** by limiting its use.
    - Select **API restriction** and check all the APIs in the dropdown for now. 
    
</figure>
<img src="https://www.dropbox.com/s/ked2jfofqpqqpab/Screen%20Shot%202023-02-19%20at%204.37.33%20PM.png?dl=1" alt="drawing" width="1000" style="display: block; margin: 0 auto"/>
</figure>



In [ ]:
api_key ='INSERT_YOUR_KEY_HERE'

## 1.2 Installing the `googlemaps` library
Here, the `-U` flag stands for installing the latest version of the library.


In [ ]:
!sudo pip3 install -U googlemaps

## 1.3 Geocoding
[Google Geocoding API documentation](https://developers.google.com/maps/documentation/geocoding/overview)

In [ ]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=api_key)

In [ ]:
# Geocoding an address
geocode_result = gmaps.geocode("921 University Ave, Ithaca, NY 14853")

In [ ]:
geocode_result

We were returned a JSON, which is just a nested dictionary of key-value pairs. 

In its simplest version, this is how JSONs and dictionaries are structured: 

</figure>
<img src="https://www.softwaretestinghelp.com/wp-content/qa/uploads/2017/12/Including-Car-in-Employee-JSON.jpg" alt="drawing" width="600" style="display: block; margin: 0 auto"/>
</figure>

What "nested" means is just that a the value associated with a key can be a dictionary itself. 
</figure>
<img src="https://help.rapid7.com/insightidr/content/resources/images/json/nested-json.png" alt="drawing" width="600" style="display: block; margin: 0 auto"/>
</figure>



Let's dig into the part of the JSON that we actually care about, which is that `'geometry': {'location': {'lat': 42.4512236, 'lng': -76.4828622},` section. 

In [ ]:
### That was a  list, so let's get the first value
geocode_result[0]

In [ ]:
geocode_result[0]['geometry']['location']

In [ ]:
print(geocode_result[0]['geometry']['location']['lng'],
     geocode_result[0]['geometry']['location']['lat'])

Now that we have the lng/lat, you can see how we can turn these into geometries. 

## 1.2.1 Geocoding a df
Let's use an example that has multiple rows. The geocoder only processes 1 request at a time, so we have to run the geocoder on each row of a df. 

Let's take a look at the Department of Building's Permit Issuance dataset. The DOB requires a permit for construction and demolition activities. 

In [ ]:
dob_permits_url = 'https://data.cityofnewyork.us/resource/ipu4-2q9a.json?$limit=5'
dob_permits= pd.read_json(dob_permits_url)

In [ ]:
dob_permits

See that we have different columns that refer to the 
- House number
- Street Name
- (Optionally: Borough)

We know all the address are in NYC. All of our relevant columns need to be strings in order for us to concatenate these columns together. 


In [ ]:
dob_permits.dtypes

In [ ]:
dob_permits['house__'] = dob_permits['house__'].astype(str)
dob_permits['street_name'] = dob_permits['street_name'].astype(str)
dob_permits['borough'] = dob_permits['borough'].astype(str)

In [ ]:
dob_permits['address'] = dob_permits['house__'] + ' ' + \
                        dob_permits['street_name'] + ', ' + \
                        dob_permits['borough'] + ', ' + 'NY'

In [ ]:
dob_permits['address']

Let's first create two new columns: `lng` and `lat`

In [ ]:
dob_permits['lng'] = ""
dob_permits['lat'] = ""

In [ ]:
for i, row in dob_permits.iterrows(): 
    print(f"Current row is {i}")
    row_result = gmaps.geocode(row['address'])

    ## We are going to assign the geocoded lat/lng to each row
    dob_permits.loc[i,'lat'] = row_result[0]['geometry']['location']['lat']
    dob_permits.loc[i,'lng'] = row_result[0]['geometry']['location']['lng']
    

In [ ]:
dob_permits

In [ ]:
## 
dob_permits_geo = gpd.GeoDataFrame(dob_permits,
                            geometry=gpd.points_from_xy(dob_permits['lng'], dob_permits['lat']),
                            crs='EPSG:4326')

In [ ]:

dob_permits_geo.explore()



## Q.1 Geocoding
- Create a DF of any five places you have visited or traveled to across the world 
    - Try inserting some typos, alternative spellings or names for these places. (For ex, Sibley Hall instead of 921 University Ave.)
- Geocode these using the google maps API. 
- Map these five points, using contextily as a basemap service. 
- How accurate was the geocoder in finding the locations of these places? 

In [ ]:
## INSERT YOUR CODE HERE

## 1.2.2 Reverse geocoding
In the same way that you can turn addresses to Lng/Lats, you can do the same. 

In [ ]:
## Note, that the format here is Lat, Lng!
reverse_geocode_result = gmaps.reverse_geocode((42.4512236,-76.4828622))


In [ ]:
reverse_geocode_result

In [ ]:
## Ta da!
reverse_geocode_result[0]['formatted_address']

## 1.3 Getting Points of Interest
[Google Places API documentation](https://developers.google.com/maps/documentation/places/web-service/overview)


In [ ]:
### The query can be anything you would typically type into
### the google maps search
### radius is in meters
### You can also input the lat/lng directly

places_to_eat = gmaps.places("restaurants",
                             location = "921 University Ave, Ithaca, NY",
                            #  location = (42.4512236,-76.4828622),
                         radius=100)

In [ ]:
places_to_eat.keys()

In [ ]:
len(places_to_eat['results'])

In [ ]:
places_to_eat['results']

In [ ]:
places_to_eat['results'][0]

In [ ]:
list_of_restaurants = []

for each in places_to_eat['results']: 
    listing_values = [each['name'],
                      each['geometry']['location']['lng'], 
                      each['geometry']['location']['lat'],
                        each['rating'],
                        each['user_ratings_total'],
                        each['photos'][0]['photo_reference']]
    list_of_restaurants.append(listing_values)

In [ ]:
# list_of_restaurants

In [ ]:
places_to_eat_df = pd.DataFrame(list_of_restaurants,columns=['name','lng','lat','rating','user_ratings_total','photo_reference'])

In [ ]:
places_to_eat_df

In [ ]:
places_to_eat_df_geo = gpd.GeoDataFrame(places_to_eat_df,
                                        geometry=gpd.points_from_xy(places_to_eat_df['lng'], places_to_eat_df['lat']),
                                        crs='EPSG:4326')

In [ ]:
places_to_eat_df_geo.explore()


## 1.4 Getting a route and distances
[Google Directions API documentation](https://developers.google.com/maps/documentation/directions)

In [ ]:
now = datetime.now()

In [ ]:
now

In [ ]:
## mode can be driving, walking, bicycling, transit
now = datetime.now()


directions_result = gmaps.directions("921 University Ave, Ithaca, NY",
                                     "Maru Ramen, Ithaca, NY",
                                     mode="driving",
                                     departure_time=now)

In [ ]:
## Notice that this is a list of one elemment
directions_result

In [ ]:
len(directions_result)

We can use the `.keys()` 

In [ ]:
directions_result[0].keys()

Neat! We can see that this has output a lot of different kinds of information. What we'll need next is to parse the information. 

Notice that the directions are in `'legs'`. This is the equivalent to each step in Google Map directions. We'll have to put the pieces together to get any information about the whole route. 



In [ ]:
## Let's access the first/only element
## and then the values in the legs key
## which is another list
directions_result[0]['legs']

In [ ]:
len(directions_result[0]['legs'])

In [ ]:
directions_result[0]['legs'][0]['steps']

We can get the total time and distance that it took: 
- `directions_result[0]['legs'][0]['steps'][i]['distance']` gives us the distance of that leg
- `directions_result[0]['legs'][0]['steps'][i]['duration']` gives us the duration of that leg

In [ ]:
distance_list = []
duration_list = []
for each in directions_result[0]['legs'][0]['steps']: 
    ## Here we are going to get the 'value' after the 'distance' key. 
    ## For the distance, this is given in meters
    # # The duration is in seconds.  
    distance_list.append(each['distance']['value'])
    duration_list.append(each['duration']['value'])

In [ ]:
print(distance_list)
print(duration_list)

Now, we can create a df, where each row could include the distance, duration, and (eventually) the route geometry

First, we'll need a list with the sums of each of the `duration_list` and `distance_list`. 

In [ ]:
## Note that, because pandas takes each element in the list as a row,
## We need to wrap the list in another list
summed_route_data = [[sum(distance_list),sum(duration_list)]]


In [ ]:
summed_route_data

Then, we'll need a list of the respective column names

In [ ]:
col_names = ['distance','duration']

In [ ]:
routes = pd.DataFrame(summed_route_data,
                      columns=col_names)

In [ ]:
routes

Turning the polyline string into points. Note that Google Maps includes an encoded version of, in this case, a polyline, which we can turn back into a shapely geometry. 

I found this function online to help me do this. `get_linestring()` takes a Google encoded polyline and turns it into a geometry, using the `decode_polyline()` function. Don't worry too much about what `decode_polyline()` is doing, except that some helpful person has created a decoder for us to use. 

In [ ]:
import json
import fiona
import pandas as pd
from shapely.geometry import LineString, mapping, shape

def decode_polyline(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs'''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index+=1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lng / 100000.0, lat / 100000.0))

    return coordinates

def get_linestring(data):
    # with open(trip_name + '.json', 'r') as data_file:    
    #     data = json.load(data_file, encoding='ISO-8859-1')

    the_points = []
    for step in data[0]['legs'][0]['steps']:
        the_points += decode_polyline(step['polyline']['points'])

    return LineString(the_points)


In [ ]:
## Here, I am getting the linestring from the directions result
geom = shape(mapping(get_linestring(directions_result)))

In [ ]:
geom

In [ ]:
routes['geom']=geom

In [ ]:
routes_geo =gpd.GeoDataFrame(routes, geometry='geom', crs='EPSG:4326')

In [ ]:
routes_geo

Chaning to 3857 because it looks better. 

In [ ]:
routes_geo_3857 = routes_geo.to_crs(epsg=3857)

In [ ]:
routes_geo_3857.explore()


## Q.2 Routing 
- Create a geodataframe with one row that describes the walking route from the **Johnson Art Museum** to the **Purity Ice Cream Co.**
- This gdf should have the following columns: start lng,start lat, start name, end lng, end lat, end name, geometry for the route, walking distance in kilometers, walking duration in minutes.

In [ ]:
## INSERT YOUR CODE HERE

## 1.4 Billing
Note that the Geocoder is **billed by usage**. Here is the pricing model: 
</figure>
<img src="https://www.dropbox.com/s/jqmy92vncaar573/Screen%20Shot%202023-02-19%20at%205.50.16%20PM.png?dl=1" alt="drawing" width="1000" style="display: block; margin: 0 auto"/>
</figure>


Think carefully about what you are querying and pay attention to your usage. Don't use up all your credits!

# 2. OPTIONAL: `beautifulsoup`

Webscraping is a method of programmatically retrieving data from websites. It is used when there are no APIs or when data that is published from the website isn't made avaliable in a downloadable format. 

#### A note on webscrapping (and why I generally avoid it):
Webscrapping can be used for all kinds of malicious purposes, for instance, to copy website content and republish it. Here's a [complaint from Craiglist](https://www.scribd.com/doc/313832868/CraigslistVRadpad-Complaint?secret_password=7gTybamKvrbeVhxfi4mx) about a company called Radpad scraping Craigslist and reposting those listing on their own website:

<mark>
“[The content scraping service] would, on a daily basis, send an army of digital robots to craigslist to copy and download the full text of millions of craigslist user ads. [The service] then indiscriminately made those misappropriated listings available—through its so-called ‘data feed’—to any company that wanted to use them, for any purpose. Some such ‘customers’ paid as much as $20,000 per month for that content…”</mark>
<br>
<br>

<mark>
According to the claim, scraped data was used for spam and email fraud, among other activities: </mark>
<br>
<br>

<mark>
“[The defendants] then harvest craigslist users’ contact information from that database, and initiate many thousands of electronic mail messages per day to the addresses harvested from craigslist servers…. [The messages] contain misleading subject lines and content in the body of the spam messages, designed to trick craigslist users into switching from using craigslist’s services to using [the defenders’] service…”
</mark>
<br>
<br>

Uff. 

**What about webscrapping for research or academic purposes?** Most of the above issues most likely won't apply to you, but webscrapping makes a website's traffic *spike* if you don't modulate how often you're pinging the website. This can cause the website's server to crash. This is not very nice. Also, a lot of websites won't allow you to do it. (If you go to almost any URL and put `/robots.txt` after it, you can see a list of subdomains that site will or won't allow you to scrape.)

The first step is to install Beautiful Soup. You can install it using pip by running the following command in your terminal:

In [ ]:
!pip install beautifulsoup4

Once you have installed Beautiful Soup, the next step is to import the required libraries in your Python script:

In [ ]:
from bs4 import BeautifulSoup
import requests

Here, we are importing BeautifulSoup from the bs4 library, which we will use to parse the HTML data we scrape from a website. We are also importing `requests`, which we will use to send HTTP requests to the website and retrieve the HTML data.

The next step is to send a request to the website and retrieve the HTML data. You can do this using the `requests.get()` method, which takes the website URL as a parameter and returns the HTML content of the website.

In [ ]:
## This is the SF Craigslist page for apartments. 
url = 'https://www.century21.com/real-estate/ithaca-ny/LCNYITHACA/'
response = requests.get(url)


Once we have retrieved the HTML data, the next step is to parse it using Beautiful Soup. We can do this by creating a BeautifulSoup object and passing the HTML data as a parameter.

The second parameter `'html.parser'` specifies the parser to use for parsing the HTML data. In this case, we are using the built-in HTML parser that comes with Beautiful Soup.



In [ ]:
soup = BeautifulSoup(response.text, 'html.parser',)


We can print soup and see what it looks like. It's essentially all the HTML served on a website: 

In [ ]:
soup

Once we have parsed the HTML data, we can use Beautiful Soup to find the data we need. We can do this by using various methods provided by Beautiful Soup, such as `find()`, `find_all()`, and `select()`, to locate the HTML elements that contain the data we need.

For example, if we want to find all the links on the website, we can use the `find_all()` method to find all the `<a>` tags on the page and then extract the href attributes:

In [ ]:
prices = []
for link in soup.find_all('a', class_= 'listing-price'):
    prices.append(link.text.replace(' ',"").replace('\n',""))

beds = []
for link in soup.find_all('div', class_= 'property-beds'):
    beds.append(link.text.replace(' ',"").replace('\n',"").replace('beds',''))

sqfts = []
for link in soup.find_all('div', class_= 'property-sqft'):
    sqfts.append(link.text.replace(' ',"").replace('\n',"").replace('sq.ft','').replace(',',''))

addresses = []
for link in soup.find_all('div', class_= 'property-address'):
    addresses.append(link.text.replace('               ',"").replace('            ',"").replace('\n',''))


In [ ]:
prices

In [ ]:
beds

In [ ]:
sqfts

In [ ]:
addresses